In [1]:
# Reference
# https://yurimkoo.github.io/python/2019/09/14/connect-db-with-python.html

import pandas as pd
import requests
import pymysql
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine
from functions.pubgapi import get_tournament_info
from functions.pubgapi import get_match_info
from functions.pubgapi import get_match_participant_single

# Load api key & RDS info (endpoint="rds 엔드포인트", dbname="db 이름", username="마스터 사용자 이름", password="rds 비밀번호")
load_dotenv()
api_key = os.environ.get("API_KEY")
endpoint = os.environ.get("AWS_endpoint")
dbname = os.environ.get('AWS_dbname')
username = os.environ.get("AWS_username")
password = os.environ.get("AWS_password")

In [2]:
# Tournament id 값 조회
tournament_info = get_tournament_info(api_key)
tournament_id = tournament_info["id"]

print(f"{len(tournament_id)} 개의 tournament_id 조회 완료\n")

# tournament_index를 지정하여 id값 출력
idx1 = 0
tournament_index_id = tournament_id[idx1]

cur_match_info = get_match_info(api_key, tournament_index_id)
num_match = len(cur_match_info["matchId"])
print(f"Tournament name: {tournament_index_id} ({num_match} matches)")

# Match participant 조회
idx2 = 2
match_info_id = cur_match_info["matchId"]
match_info_id_single = match_info_id[idx2]
match_participant = get_match_participant_single(api_key, match_info_id_single)
match_participant

466 개의 tournament_id 조회 완료

Tournament name: eu-trt22 (9 matches)


,player_id,team_rank,dbnos,assists,boosts,damage_dealt,headshot_kills,heals,kill_streaks,kills,longest_kill,revives,team_kills,time_survived
0,MIT_Hueth,1,3,3,11,251.031600,1,2,1,1,46.858997,0,0,1817
1,MIT_Rid1,1,3,2,6,458.498260,0,3,2,3,41.820904,0,0,1747
2,MIT_Iroh,1,2,3,5,559.220200,0,1,2,2,36.970795,0,0,1772
3,MIT_Bluffist,1,2,0,3,255.048400,2,1,1,2,253.195470,0,0,1568
4,MIT_forge2k,1,0,1,4,126.593544,0,3,1,1,318.092040,0,0,1799
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,10K_ToygaaR,3,3,0,5,93.522630,0,7,2,2,170.119280,0,0,1743
60,10K_DutchVelders,3,2,3,17,368.500270,0,3,2,2,15.159199,0,0,1737
61,10K_Resqperz,3,3,3,11,461.262600,0,3,1,2,58.741665,0,0,1792
62,10K_Peniag,3,2,1,5,189.789060,0,3,1,1,136.303270,0,0,1737


In [3]:
# Kill 수의 합이 0인 경우 제외(연습게임)
if match_participant["kills"].sum() != 0:
    upload_table = match_participant
    upload_table_name = f"{match_info_id_single}"

In [5]:
# DB와 connect
# create_engine("mysql+pymysql://아이디:"+"암호"+"@주소:포트/데이터베이스이름?charset=utf8", encoding='utf-8')
engine = create_engine("mysql+pymysql://admin:"+password+"@"+endpoint+":3306/"+dbname+"?charset=utf8", encoding="utf-8")
conn_create = engine.connect()

# AWS RDS에 tournament_info 테이블 create
upload_table.to_sql(name=upload_table_name, con=conn_create, if_exists='replace', index=False)
conn_create.close()

# DB connect
conn = pymysql.connect(host=endpoint, user=username, password=password, db=dbname, charset='utf8', port=3306)
cur = conn.cursor()

# 업로드된 테이블 확인(SQL 사용)
sql = f"SHOW TABLES LIKE '{upload_table_name}';"
cur.execute(sql)
result = cur.fetchall()
print("SHOW TABLES: \n",result)

# 내용 확인(SELECT *)
sql = f"SELECT * FROM {upload_table_name};"
cur.execute(sql)
result1 = cur.fetchall()
result1 = pd.DataFrame(result1)
print("\ntournament_info(database):\n",result1)

SHOW TABLES: 
 (('49634508-f42a-4c39-a696-360347083736',),)


ProgrammingError: (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '49634508-f42a-4c39-a696-360347083736' at line 1")

In [ ]:
conn.commit()

# db접속 종료
conn.close()